In [ ]:
%matplotlib inline
import pandas as pd 
import numpy as np

import torch 
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

from modules.classifer_utils import ClassifierDataset, TrainingManager

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")





In [ ]:

file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
heart_csv_df = pd.read_csv(file_url)

heart_csv_df

In [ ]:
## TODO consider skip last!


[encoded_df] = ClassifierDataset.onehot_encode_datafames( [heart_csv_df] )

overall_ds = ClassifierDataset(encoded_df, "target")

train_ds, test_ds = random_split(overall_ds, [.75, .25])

batch_size = int(len(train_ds) / 10)
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

print(f'{len(train_ds)} training records in with batch size {batch_size}, {len(test_ds)} records for test')

first_training_record, _ = train_ds[0]
num_features = first_training_record.shape[-1]
print(f'datasets have {num_features} features')


In [ ]:
DROPOUT_RATE_01 = .2

class HeartRateNN(nn.Module):
    def __init__(self, num_feature_columns):
        super().__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(num_feature_columns, 32),
            nn.Dropout(p=DROPOUT_RATE_01), 
            nn.GELU(),
            nn.Linear(32, 16),
            nn.Dropout(p=DROPOUT_RATE_01), 
            nn.GELU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.linear_stack(x)
    

first_training_record, _ = train_ds[0]
num_features = first_training_record.shape[-1]

model = HeartRateNN(num_features)
print(model)

In [ ]:


SEED = 123
torch.manual_seed(SEED)


if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

first_training_record, _ = train_ds[0]
num_feature_columns = first_training_record.shape[-1]

model = HeartRateNN( num_feature_columns )
model.to(device)

loss_fn   = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 200

for epoch in range(num_epochs):
    epoch_correct_count, epoch_pred_count = 0, 0
    for X, y in train_dataloader:

        X = X.to(device)
        y = y.to(device)

        y_pred = model(X)
        y_pred = y_pred.reshape(y.shape)
        loss = loss_fn(y_pred, y)

        y_pred_guess = torch.round(y_pred)
        batch_num_correct = (y == y_pred_guess).sum()
        epoch_correct_count += batch_num_correct
        epoch_pred_count += len(y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], {epoch_correct_count} of {epoch_pred_count} correct {(100*epoch_correct_count/epoch_pred_count):.1f} %")



In [ ]:

training_mgr = TrainingManager()
training_mgr.eval(model, test_dataloader)
